chargement et preparation donnees

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import pandas as pd

# Chemin du fichier CSV
csv_file = "/content/sakila_combined.csv"

# Liste des noms à attribuer aux colonnes
noms_colonnes = [
    "film_id", "film_title", "film_description", "release_year",
    "rental_rate", "film_length", "category_name", "customer_id", "customer_name", "customer_email",
    "staff_id", "staff_name", "store_id", "store_city", "store_country", "rental_id", "rental_date", "return_date", "payment_amount"
]

# Lire le CSV sans noms de colonnes et attribuer les noms
df = pd.read_csv(csv_file, header=None, names=noms_colonnes)

In [ ]:
# Afficher un aperçu du DataFrame
print("Aperçu du fichier avec les colonnes attribuées :")
print(df.head())


Aperçu du fichier avec les colonnes attribuées :
   film_id    film_title                                   film_description  \
0       19  AMADEUS HOLY  A Emotional Display of a Pioneer And a Technic...   
1       19  AMADEUS HOLY  A Emotional Display of a Pioneer And a Technic...   
2       19  AMADEUS HOLY  A Emotional Display of a Pioneer And a Technic...   
3       19  AMADEUS HOLY  A Emotional Display of a Pioneer And a Technic...   
4       19  AMADEUS HOLY  A Emotional Display of a Pioneer And a Technic...   

   release_year  rental_rate  film_length category_name  customer_id  \
0          2006         0.99          113        Action           77   
1          2006         0.99          113        Action           39   
2          2006         0.99          113        Action          254   
3          2006         0.99          113        Action           89   
4          2006         0.99          113        Action           36   

    customer_name                     custo

In [ ]:

# Conversion des colonnes de dates
df['rental_date'] = pd.to_datetime(df['rental_date'])
df['return_date'] = pd.to_datetime(df['return_date'])

In [ ]:

# Calculer la durée de la location en jours
df['rental_duration'] = (df['return_date'] - df['rental_date']).dt.days

In [ ]:
# Calculer le nombre de jours depuis la dernière location pour chaque client
df['last_rental_date'] = df.groupby('customer_id')['rental_date'].transform('max')
df['last_rental_days'] = (df['last_rental_date'] - df['rental_date']).dt.days

In [ ]:
# Calculer la fréquence de location par client
frequencies = df.groupby('customer_id')['rental_id'].count().reset_index()
frequencies.columns = ['customer_id', 'frequence_location']
df = df.merge(frequencies, on='customer_id', how='left')

In [ ]:

# Moyenne du montant payé par client
avg_payment = df.groupby('customer_id')['payment_amount'].mean().reset_index()
avg_payment.columns = ['customer_id', 'avg_payment']
df = df.merge(avg_payment, on='customer_id', how='left')

In [ ]:

# Créer une variable cible
df['target'] = (df['last_rental_days'] <= 30).astype(int)

# Sélectionner les caractéristiques

In [ ]:

# Sélectionner les caractéristiques
features = ['frequence_location', 'rental_rate', 'film_length', 'category_name', 'avg_payment', 'last_rental_days']
X = df[features]
y = df['target']

In [ ]:
# Appliquer le préprocesseur pour convertir les variables catégorielles
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), ['category_name']),
        ('num', StandardScaler(), ['frequence_location', 'rental_rate', 'film_length', 'avg_payment', 'last_rental_days'])
    ])

# Séparer les données en ensembles d'entraînement et de test

In [ ]:

# Séparer les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Appliquer le préprocesseur

In [ ]:

# Appliquer le préprocesseur
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [ ]:
# Convertir les matrices creuses en matrices denses si nécessaire
X_train = X_train.toarray()  # Convertir en array dense
X_test = X_test.toarray()  # Convertir en array dense


# Créer et entraîner le modèle KNN

In [ ]:
# Créer et entraîner le modèle KNN
knn = KNeighborsClassifier(n_neighbors=5)  # Vous pouvez ajuster n_neighbors
knn.fit(X_train, y_train)

KNeighborsClassifier()

# Prédire avec le modèle

In [ ]:
# Prédire avec le modèle
y_pred = knn.predict(X_test)

In [ ]:
# Afficher les résultats de la classification
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.99      0.96      0.97      1406
           1       0.97      0.99      0.98      1767

    accuracy                           0.98      3173
   macro avg       0.98      0.97      0.98      3173
weighted avg       0.98      0.98      0.98      3173



# Exemple de prédictions sur de nouvelles données

In [ ]:
# Supposons que nous ayons un nouveau client avec les caractéristiques suivantes:
nouveau_client = {
    'frequence_location': [2],  # Le client a effectué 2 locations
    'rental_rate': [2.99],       # Le tarif de location est 2.99
    'film_length': [120],        # La longueur du film est 120 minutes
    'category_name': ['Action'], # La catégorie du film est 'Action'
    'avg_payment': [10.5],       # Le client a payé en moyenne 10.5
    'last_rental_days': [15]     # Le client a loué un film dans les 15 derniers jours
}

# Convertir en DataFrame
nouveau_client_df = pd.DataFrame(nouveau_client)

# Appliquer le préprocesseur sur les nouvelles données
nouveau_client_transforme = preprocessor.transform(nouveau_client_df)

# Convertir en array dense
nouveau_client_transforme = nouveau_client_transforme.toarray()

# Faire une prédiction
prediction = knn.predict(nouveau_client_transforme)

# Afficher la prédiction
print("La prédiction pour ce client est :", "Location probable dans les 30 jours" if prediction[0] == 1 else "Pas de location probable dans les 30 jours")

La prédiction pour ce client est : Location probable dans les 30 jours
